# Getting Started With TensorFlow

본 가이드는 텐서플로우 프로그래밍을 시작하겠습니다. 가이드에 본격적으로 들어가기 전에 먼저 텐서플로우를 설치해줍시다. [윈도우설치법](http://fabj.tistory.com/43) 본 가이드를 제대로 따르기 위해서는 아래 사랑을 숙지해야합니다. 

* 파이썬 프로그래밍
* 배열(Array)에 대한 기초적인 이해
* 머신러닝에 대한 이해, 하지만 머신러닝에 대한 이해가 부족하거나, 없더라도 본가이드는 여전히 여러분의 첫번째 가이드가 될 수 있습니다. 

텐서플로우는 여러 API를 제공합니다. 가장 낮은 단의 API(Tensorflow Core)는 여러분에게 완벽한 프로그래밍을 지원합니다. 우리는 머신러닝 연구자와 모델에 대한 파인튜닝에 대해 Tensorflow Core 사용을 권장합니다. 고수준(High Level) API는 Tensorflow Core를 기반으로 만들어졌습니다. 고수준 API는 Tensorflow Core 보다 쉽게 다룰수 있습니다. 또한, 반복적인 작업을 보다 쉽고, 일관으로 만듭니다. **tf.contrib.learn**과 같은 고수준 API를 통해 manage data sets, estimators, training, inference 작업을 쉽게 할 수 있습니다. 고수준 API는 여전히 개발 중에 있음을 숙지하기바랍니다. 텐서플로우의 새버전이 릴리즈 될 때 기존 매써드(method)는 바뀌거나, 없어 질 수 있습니다. 

본 가이드는 TensorFlow Core의 튜토리얼로 시작합니다. 그리고 같은 모델이 tf.contrib.learn으로 어떻게 구현되는지 보여 줄 것입니다. TensorFlow Core에 대한 기본적인 이해는 여러분이 고수준 API를 사용할 때 내부 매커니즘이 어떻게 동작하는지 아는데 도움을 줍니다. 

# Tensors

TensorFlow의 가장 중요한 데이터유닛은 tensor입니다. tensor는 다차원 배열의 형태를 띄는 primitive값으로 구성됩니다. tensor의 rank는 tensor를 구성하는 배열의 차원과 같습니다. 여기 tensor에 대한 몇가지 예시가 있습니다.
```python
3 # a rank 0 tensor; this is a scalar with shape []
[1. ,2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```
## TensorFlow Core tutorial

### Importing TensorFlow

TensorFlow 프로그래밍에서 표준 import문은 아래와 같습니다. 


In [1]:
import tensorflow as tf

이로써 Python에서 TensorFlow의 모든 class, method, symbol에 접근 할 수 있습니다. 대부분 가드문에서는 이 부분은 이미 작성되었다 가정합니다.

### The Computational Graph

TensorFlow Core 프로그래밍은 두 개별 섹션으로 구성되어 있다고 생각할 수 있습니다.

1. computational graph 만들기.
2. computational graph 동작(Run).

**computational graph**는 나열된 node에 대한 TensorFlow 연산으로 볼 수 있습니다. 간단한 computational graph를 만들어 보겠습니다. 각 노드는 0개 이상 입력텐서를 가지며, tensor를 출력합니다.node의 type 중 하나는 constant입니다. TensorFlow constant와 같이 입력이 없으며, 저장하고 있는 값을 출력합니다. 아래와 같이 floating point Tensors인 node1, node2를 만들 수 있습니다.

In [2]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


마지막 print문은 다음과 같이 출력합니다. 
```python
Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)
```
여러분의 예측과 달리 3.0과 4.0 같은 값을 출력하지 않음을 주목합시다. 대신 node1, node2는 아직 노드입니다. evaluation 후에 노드는 3.0과 4.0같은 값을 내보냅니다. node를 evaluate하려면 **session**을 통해 computational graph를 run해야합니다. session은 TensorFlow runtime의 상태와 컨트롤에 대해 캡슐화를 수행합니다.

아래 코드를 통해 Session objec를 정의하고, run method를 호출하여 computational graph를 구성하는 node1과 node2를 evaluate합니다. 아래와 같이 session 내에서 computational graph를 run합니다. 

In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


이제 일적으로 예측했던 3.0, 4.0 같은 값이 출력되었습니다.:
```python
[3.0, 4.0]
```
노드들을 연산(연산자 역시 노드입니다.)과 결합해 조금 더 복잡한 계산을 설계할 수 있습니다. 예를 들어 아래와 같이 두 constant node를 더하는 새로운 그래프를 만들 수 있습니다. 

In [4]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


마지막 두 print문은 아래와 같이 출력할 것입니다. 
```python
node3:  Tensor("Add_2:0", shape=(), dtype=float32)
sess.run(node3):  7.0
```
TensorFlow는 computational graph를 보여주는 TensorBoard라는 유틸리티를 제공합니다. 아래 그림은 TensorBoard를 통해 그린 computational graph입니다. 
![Turing's Device](https://www.tensorflow.org/images/getting_started_add.png)
사실 위 그래프는 항상 같은 값만 출력하므로 그리 흥미롭지 못합니다. placeholder를 사용하여 그래프가 외부 파라미터입력을 받도록 할 수 있습니다. placeholder는 값을 나중에 받습니다. 

In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

위 세줄은 두 입력과 연산자로 구성된 함수나 lambda처럼 보입니다. feed_dict parameter를 통해 구체적인 값을 가지는 tensor를 그래프의 입력으로 지정할 수 있으며, 따라서 주어진 그래프를 여러 입력에 대해 evaluate 할 수 있습니다. 

In [6]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


출력값은 아래와 같습니다. 
```python
7.5
[ 3.  7.]
```
TensorBoard에서는 그래프가 아래와 같이 보입니다. 
![Turing's Device](https://www.tensorflow.org/images/getting_started_adder.png)
아래와 같이 다른 연산을 추가하여 그래프를 더 복잡하게 만들 수 있습니다. 

In [7]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


출력값은 아래와 같습니다. 
```python
22.5
```
TensorBoard에서는 그래프가 아래와 같이 보일 것입니다. 
![Turing's Device](https://www.tensorflow.org/images/getting_started_triple.png)
머신러닝에서는 일반적으로 위와 같이 1개 이상의 입력을 가지는 모델을 원합니다. 모델이 학습가능하게 만들려면, 같은 입력이 들어와도 다를 출력이 나가게 그래프를 수정할 필요가 있습니다. **Variables**는 학습 가능한 파라미터를 제공합니다. 이것은 type 과 초기값으로 구성됩니다. 

In [8]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

상수는 $tf.constant$를 통해 초기화 했습니다. 그리고 그 값은 절대 변하지 않습니다. 반면, 변수는 $tf.Variable$로 초기화 됩니다. TensorFlow program내 모든 변수를 초기화 하기위해, 아래와 같이 특별한 operation를 호출해야합니다.

In [9]:
init = tf.global_variables_initializer()
sess.run(init)

모든 global variables을 초기화 하므로, **init**을 불러오는 것은 중요합니다. **sess.run**을 호출하기 전까지는 변수들은 초기화 되지 않습니다. 
**x**는 placeholder기 때문에, 아래와 같이 여러 x값에 대해 **linear_model**을 동시에 evaluate 할 수 있습니다. 

In [10]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


아래와 같이 출력됩니다. 
```python
[ 0.          0.30000001  0.60000002  0.90000004]
```

우리는 모델을 만들었지만 아직 얼마나 좋은지 모릅니다. 학습 데이터에 대한 모델을 평가하려면 목표 값을 제공하기 위해 y placeholder 필요하며 손실(loss) 함수를 작성해야합니다.

손실 함수는 제공된 데이터로부터 현재 모델이 얼마나 차이나는지를 측정합니다. 현재 모델과 제공된 데이터 사이의 델타의 제곱을 합으로 표현되는 선형 회귀에대한 표준 손실 모델을 사용합니다. **linear_model - y**는 각 요소가 위 예제의 오차 델타 벡터를 만듭니다. **tf.square**를 호출하여 오류를 제곱합니다. 그런 다음 모든 제곱 된 오류를 합하여 ** tf.reduce_sum **을 사용하여 모든 예제의 오류를 표현하는 단일 스칼라를 만듭니다.

In [11]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


loss 값은 아래와 같습니다. 
```python
23.66
```
** W **와 ** b **의 값을 목표값인 -1과 1을 재할당하여 수동으로 향상시킬 수 있습니다. 변수는 ** tf.Variable **에 제공된 값으로 초기화되지만 **tf.assign**과 같은 operation을 사용하여 변경되었습니다. 예를 들어, ** W = -1 ** 및 ** b = 1 **는 위 모델에 대한 최적의 파라미터입니다. ** W ** 및 ** b **를 적절한 값으로 변경할 수 있습니다.

In [12]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


loss가 0으로 출력됨을 볼 수 있습니다. 
```python
0.0
```
** W ** 및 ** b **의 "완벽한"값을 추측했지만 머신러닝의 요점은 모델의 올바른 파라미터를 자동으로 찾는 것입니다. 다음 섹션에서 이를 수행하는 방법을 보여줄 것입니다.

### tf.train API

머신러닝에 대한 자세한 내용은 이 튜토리얼의 범위를 벗어납니다. 그러나 TensorFlow는 손실 함수를 최소화하기 위해 각 변수를 조금씩 변경하는 **옵티마이저**를 제공합니다. 가장 간단한 옵티마이저는 그래디언트 디센트입니다. 해당 변수에 대한 loss의 derivative의 크기에 따라 각 변수를 수정합니다. 일반적으로 symbolic derivative를 수동으로 계산하는 것은 지루하며, 오류가 발생하기 쉽습니다. 결과적으로 TensorFlow는 **tf.gradients** 함수를 사용하여 모델에 제공된 derivative를 자동으로 생성 할 수 있습니다. 단순화를 위해 일반적으로 옵티마이저가 이를 수행합니다. 예를 들어, 아래와 같습니다. 

In [13]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


최종 모델의 파라미터는 다음과 같습니다. 
```python
[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]
```
이제 실제 머신러닝을 했습니다! 이 간단한 선형 회귀를 수행하는데는 TensorFlow Core 코드가 많이 필요하지는 않지만, 모델에 여러 데이터를 입력하는 더 복잡한 모델과 메서드는 더 많은 코드가 필요합니다. 따라서 TensorFlow는 일반적인 패턴, 구조 및 기능에 대해 더 높은 수준의 추상화를 제공합니다. 우리는 다음 절에서 이러한 추상화를 사용하는 방법을 배웁니다.

### Complete program

훈련 가능한 선형 회귀 모델의 전체 코드는 아래와 같습니다. 

In [14]:
import numpy as np
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


run후의 결과는 아래와 같습니다. 
```python
W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
```
완성된 전체 그래프는 TensorBoard에 다음과 같이 그려집니다.  
![Turing's Device](https://www.tensorflow.org/images/getting_started_final.png)